In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv(r'C:\Users\hp\Documents\GitHub\Forecast_Treasury_Curve\Dataset\final_feature_library_all_features.csv')

In [4]:
data['Spread'] = data['USGG10YR_mean'] - data['USGG2YR_mean']

# ARIMA Model

In [6]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
from itertools import product
import matplotlib.pyplot as plt

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Assuming 'data' is your DataFrame with 'Spread' column
# Define test size
test_size = 36

# Split data into train and test
train_data = data[:-test_size].copy()
test_data = data[-test_size:].copy()
train_series = train_data['Spread']
test_series = test_data['Spread']

print(f"Training set length: {len(train_data)}")
print(f"Test set length: {len(test_data)}")

# Check stationarity
def check_stationarity(series, series_name="Series"):
    """
    Perform Augmented Dickey-Fuller test to check stationarity
    """
    result = adfuller(series.dropna())
    print(f'\n{series_name} - Augmented Dickey-Fuller Test:')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.6f}')
    
    if result[1] <= 0.05:
        print("Series is stationary (reject null hypothesis)")
        return True
    else:
        print("Series is non-stationary (fail to reject null hypothesis)")
        return False

# Check stationarity of original series
is_stationary = check_stationarity(train_series, "Original Training Series")

# Define parameter ranges for grid search
p_values = range(0, 6)  # AR order
d_values = range(0, 3)  # Differencing order
q_values = range(0, 6)  # MA order

# Create all combinations
param_combinations = list(product(p_values, d_values, q_values))
print(f"\nTotal parameter combinations to test: {len(param_combinations)}")

def fit_arima_model(series, order):
    """
    Fit ARIMA model with given order
    Returns fitted model or None if fitting fails
    """
    try:
        model = ARIMA(series, order=order)
        fitted_model = model.fit()
        return fitted_model
    except Exception as e:
        return None

def calculate_information_criteria(fitted_model):
    """
    Calculate AIC, BIC, and HQIC for model selection
    """
    try:
        return {
            'AIC': fitted_model.aic,
            'BIC': fitted_model.bic,
            'HQIC': fitted_model.hqic
        }
    except:
        return {
            'AIC': np.inf,
            'BIC': np.inf,
            'HQIC': np.inf
        }

def arima_time_series_cv(train_data, test_data, order, min_train_size=30):
    """
    Perform time series cross-validation for ARIMA model
    """
    cv_scores = []
    n_test = len(test_data)
    
    # Combine train and test data
    full_series = pd.concat([train_data, test_data])
    train_end_idx = len(train_data)
    
    for i in range(min_train_size, n_test):
        try:
            # Use training data + first i test observations
            cv_train_series = full_series.iloc[:train_end_idx + i]
            
            # Fit ARIMA model
            model = ARIMA(cv_train_series, order=order)
            fitted_model = model.fit()
            
            # Forecast next observation
            forecast = fitted_model.forecast(steps=1)
            cv_val_actual = test_data.iloc[i]
            cv_val_pred = forecast.iloc[0] if hasattr(forecast, 'iloc') else forecast[0]
            
            mae_fold = abs(cv_val_actual - cv_val_pred)
            cv_scores.append(mae_fold)
            
        except Exception as e:
            # Skip this fold if model fitting fails
            continue
    
    return cv_scores

# Hyperparameter tuning
print(f"\n{'='*80}")
print("ARIMA HYPERPARAMETER TUNING - GRID SEARCH")
print(f"{'='*80}")

results = []
best_aic = np.inf
best_bic = np.inf
best_cv_mae = np.inf
best_params_aic = None
best_params_bic = None
best_params_cv = None

print(f"{'Order (p,d,q)':<15} {'AIC':<10} {'BIC':<10} {'HQIC':<10} {'CV MAE':<10} {'CV Folds':<10} {'Status'}")
print(f"{'-'*85}")

for i, (p, d, q) in enumerate(param_combinations):
    order = (p, d, q)
    
    # Fit model on training data
    fitted_model = fit_arima_model(train_series, order)
    
    if fitted_model is not None:
        # Calculate information criteria
        ic_scores = calculate_information_criteria(fitted_model)
        
        # Perform cross-validation
        cv_scores = arima_time_series_cv(train_series, test_series, order, min_train_size=5)
        
        if len(cv_scores) > 0:
            mean_cv_mae = np.mean(cv_scores)
            n_cv_folds = len(cv_scores)
            status = "Success"
            
            # Track best models
            if ic_scores['AIC'] < best_aic:
                best_aic = ic_scores['AIC']
                best_params_aic = order
            
            if ic_scores['BIC'] < best_bic:
                best_bic = ic_scores['BIC']
                best_params_bic = order
            
            if mean_cv_mae < best_cv_mae:
                best_cv_mae = mean_cv_mae
                best_params_cv = order
            
            results.append({
                'order': order,
                'AIC': ic_scores['AIC'],
                'BIC': ic_scores['BIC'],
                'HQIC': ic_scores['HQIC'],
                'CV_MAE': mean_cv_mae,
                'CV_Folds': n_cv_folds,
                'fitted_model': fitted_model
            })
            
            print(f"{str(order):<15} {ic_scores['AIC']:<10.2f} {ic_scores['BIC']:<10.2f} "
                  f"{ic_scores['HQIC']:<10.2f} {mean_cv_mae:<10.4f} {n_cv_folds:<10} {status}")
        
        else:
            print(f"{str(order):<15} {ic_scores['AIC']:<10.2f} {ic_scores['BIC']:<10.2f} "
                  f"{ic_scores['HQIC']:<10.2f} {'No CV':<10} {'0':<10} No CV")
    
    else:
        print(f"{str(order):<15} {'Failed':<10} {'Failed':<10} {'Failed':<10} {'Failed':<10} {'0':<10} Failed")
    
    # Progress indicator
    if (i + 1) % 20 == 0:
        print(f"Progress: {i + 1}/{len(param_combinations)} combinations tested")

# Sort results by different criteria
results_df = pd.DataFrame(results)

if len(results_df) > 0:
    print(f"\n{'='*80}")
    print("BEST MODELS BY DIFFERENT CRITERIA:")
    print(f"{'='*80}")
    
    # Best by AIC
    best_aic_result = results_df.loc[results_df['AIC'].idxmin()]
    print(f"Best by AIC: {best_aic_result['order']} (AIC: {best_aic_result['AIC']:.2f})")
    
    # Best by BIC
    best_bic_result = results_df.loc[results_df['BIC'].idxmin()]
    print(f"Best by BIC: {best_bic_result['order']} (BIC: {best_bic_result['BIC']:.2f})")
    
    # Best by CV MAE
    best_cv_result = results_df.loc[results_df['CV_MAE'].idxmin()]
    print(f"Best by CV MAE: {best_cv_result['order']} (CV MAE: {best_cv_result['CV_MAE']:.4f})")
    
    # Top 5 models by each criterion
    print(f"\n{'='*80}")
    print("TOP 5 MODELS BY AIC:")
    print(f"{'='*80}")
    top_aic = results_df.nsmallest(5, 'AIC')[['order', 'AIC', 'BIC', 'CV_MAE']]
    print(top_aic.to_string(index=False))
    
    print(f"\n{'='*80}")
    print("TOP 5 MODELS BY BIC:")
    print(f"{'='*80}")
    top_bic = results_df.nsmallest(5, 'BIC')[['order', 'AIC', 'BIC', 'CV_MAE']]
    print(top_bic.to_string(index=False))
    
    print(f"\n{'='*80}")
    print("TOP 5 MODELS BY CV MAE:")
    print(f"{'='*80}")
    top_cv = results_df.nsmallest(5, 'CV_MAE')[['order', 'AIC', 'BIC', 'CV_MAE']]
    print(top_cv.to_string(index=False))
    
    # Final model selection (prioritize CV performance)
    final_best_order = best_cv_result['order']
    final_best_model = best_cv_result['fitted_model']
    
    print(f"\n{'='*80}")
    print("FINAL SELECTED MODEL:")
    print(f"{'='*80}")
    print(f"Selected ARIMA order: {final_best_order}")
    print(f"AIC: {best_cv_result['AIC']:.2f}")
    print(f"BIC: {best_cv_result['BIC']:.2f}")
    print(f"CV MAE: {best_cv_result['CV_MAE']:.4f}")
    print(f"Number of CV folds: {best_cv_result['CV_Folds']}")
    
    # Model summary
    print(f"\nModel Summary:")
    print(final_best_model.summary())
    
    # Generate forecasts for test set
    print(f"\n{'='*80}")
    print("TEST SET PERFORMANCE:")
    print(f"{'='*80}")
    
    # Forecast test set
    test_forecasts = final_best_model.forecast(steps=test_size)
    test_mae = mean_absolute_error(test_series, test_forecasts)
    test_rmse = np.sqrt(mean_squared_error(test_series, test_forecasts))
    
    print(f"Test MAE: {test_mae:.4f}")
    print(f"Test RMSE: {test_rmse:.4f}")
    
    # Show first 10 forecasts
    print(f"\nFirst 10 Test Forecasts:")
    forecast_comparison = pd.DataFrame({
        'Actual': test_series.iloc[:10].values,
        'Forecast': test_forecasts[:10],
        'Error': np.abs(test_series.iloc[:10].values - test_forecasts[:10])
    })
    print(forecast_comparison.round(4))
    
    # Function to get the best model
    def get_best_arima_model():
        return final_best_model, final_best_order
    
    # Save results for later use
    arima_results = {
        'best_model': final_best_model,
        'best_order': final_best_order,
        'test_mae': test_mae,
        'test_rmse': test_rmse,
        'all_results': results_df
    }
    
    print(f"\nARIMA hyperparameter tuning completed!")
    print(f"Best model: ARIMA{final_best_order}")
    
else:
    print("No successful ARIMA models fitted. Check your data and parameter ranges.")

Training set length: 267
Test set length: 36

Original Training Series - Augmented Dickey-Fuller Test:
ADF Statistic: -3.195596
p-value: 0.020241
Critical Values:
	1%: -3.455953
	5%: -2.872809
	10%: -2.572775
Series is stationary (reject null hypothesis)

Total parameter combinations to test: 108

ARIMA HYPERPARAMETER TUNING - GRID SEARCH
Order (p,d,q)   AIC        BIC        HQIC       CV MAE     CV Folds   Status
-------------------------------------------------------------------------------------
(0, 0, 0)       696.64     703.81     699.52     1.5059     31         Success
(0, 0, 1)       358.21     368.97     362.53     0.7578     31         Success
(0, 0, 2)       112.26     126.61     118.02     0.4389     31         Success
(0, 0, 3)       4.81       22.75      12.01      0.3141     31         Success
(0, 0, 4)       -94.08     -72.56     -85.44     0.2578     31         Success
(0, 0, 5)       -142.65    -117.54    -132.56    0.1995     31         Success
(0, 1, 0)       -254.